# Anime Recommendation System

- ณัฐพนธ์ สุขถาวร 62010278
- นิธิ น้อมประวัติ 62010497
- พักตร์ภูมิ ตาแพร่ 62010609
- สิรวิชญ์ สุขวัฒนาวิทย์ 62010948

## Import Libraries

In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Import Datasets

In [20]:
ANIME_CSV_PATH = "./datasets/anime.csv"
anime_df = pd.read_csv(ANIME_CSV_PATH)

In [21]:
RATING_CSV_PATH = "./datasets/rating.csv"
rating_df = pd.read_csv(RATING_CSV_PATH)

In [35]:
anime_df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [36]:
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12017 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12017 non-null  int64  
 1   name      12017 non-null  object 
 2   genre     12017 non-null  object 
 3   type      12017 non-null  object 
 4   episodes  12017 non-null  object 
 5   rating    12017 non-null  float64
 6   members   12017 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 751.1+ KB


In [24]:
rating_df

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [25]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


## Data Exploration

In [31]:
merged_df = rating_df.merge(anime_df, on='anime_id', how='left')
merged_df.rename(columns={'rating_x':'user_rating','rating_y':'avg_rating'}, inplace=True)
# merged_df['rating'] = merged_df.apply(lambda x: x['user_rating'] if x['user_rating'] > -1 else x['avg_rating'], axis=1)
# merged_df.drop(['rating_x', 'rating_y'], axis=1)
merged_df


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7813737 entries, 0 to 7813736
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      int64  
 1   anime_id     int64  
 2   user_rating  int64  
 3   name         object 
 4   genre        object 
 5   type         object 
 6   episodes     object 
 7   avg_rating   float64
 8   members      float64
dtypes: float64(2), int64(3), object(4)
memory usage: 596.1+ MB


In [32]:
# check data exploration
# sns.pairplot(merge_df, x_vars=merge_df.columns, y_vars=["rating"])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7813611 entries, 0 to 7813736
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      int64  
 1   anime_id     int64  
 2   user_rating  int64  
 3   name         object 
 4   genre        object 
 5   type         object 
 6   episodes     object 
 7   avg_rating   float64
 8   members      float64
dtypes: float64(2), int64(3), object(4)
memory usage: 596.1+ MB


## Cleaning Data

## Collaborative Filtering

## Content-based filtering

## Evaluation

### MAE (Mean Absolute Error)
- MAE = 1/n * Σ |y - y^|

### TOP-K
- Top-K = N/K
- N = 5
- K = 20

### AUC
- AUC = (n'+0.5n") / n